In [300]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame 

import matplotlib.pyplot as plt 
import seaborn as sns
import itertools
%matplotlib inline
from matplotlib.pyplot import figure
import matplotlib as mpl
import matplotlib.font_manager as fm
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'Malgun Gothic' 
plt.rcParams["font.size"] = 10
%matplotlib inline 


from datetime import datetime, timedelta 
from statsmodels.tsa.arima_model import ARIMA 

from sklearn.preprocessing import StandardScaler
import os

In [301]:
# import os
# for dirname, _, filenames in os.walk('/kaggle/input/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

## 데이터셋 1, 2 NaN %
- dataset 1(train): 77.2%
- dataset 2(test): 26.9%

In [302]:
d1 = pd.read_csv("data/train.csv", sep=',', index_col = 0)
d2 = pd.read_csv("data/test.csv", sep=',', index_col = 0)

In [303]:
d2.shape

(8760, 200)

In [304]:
sum(d1.isnull().sum().values) / (d1.shape[0] * d1.shape[1])

0.7720727241296169

In [305]:
d1.shape[0] * d1.shape[1]

21981700

In [306]:
sum(d2.isnull().sum().values) / (d2.shape[0] * d2.shape[1])

0.26900513698630135

In [307]:
d2.shape[0] * d2.shape[1]

1752000

In [308]:
sum(d2.isnull().sum().values)

471297

## Weather 추가변수
- 나중에 모델 돌릴때마다 df에 추가하는 것

In [309]:
# hourly WEATHER data
# w_hr = pd.read_csv("/kaggle/input/weather/hourly.csv",encoding='euc-kr', sep=',')
w_hr = pd.read_csv("data/weather/hourly.csv", sep=',', encoding='euc-kr')
w_hr2 = w_hr.loc[:,['Time', 'Temp','Humidity', 'WindSpeed']]
w_hr2.rename(columns={'Humidity':'Humid', 'WindSpeed':'Wind'}, inplace=True)
w_hr2['Humid']=w_hr2.loc[:,'Humid'].fillna(0)
w_hr2.head(2)

,Time,Temp,Humid,Wind
0,2016.7.20 0:00,25.7,0.0,1.3
1,2016.7.20 1:00,25.6,0.0,1.4


In [310]:
w_hr2['Time'] = pd.to_datetime(w_hr2.Time)
w_hr2 = w_hr2.set_index('Time')
w_hr2.head(2)

,Temp,Humid,Wind
Time,,,
2016-07-20 00:00:00,25.7,0.0,1.3
2016-07-20 01:00:00,25.6,0.0,1.4


## 파생 변수

In [311]:
# test = pd.read_csv("/kaggle/input/dataset/TEST_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')
test = pd.read_csv("data/TEST_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')
train = pd.read_csv("data/TRAIN_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')

In [312]:
test.index = pd.to_datetime(test.index)
train.index = pd.to_datetime(train.index)

In [313]:
def dateTimeVar(df):
    df1 = pd.DataFrame(df.index)
    df1.index = pd.to_datetime(df1.iloc[:,0])
    df1 = df1.drop(columns=df1.columns)
    # datetime을 사용한 파생변수
    # 연도, 월, 일, 요일, 시간
    df1['Mth']= df1.index.month
    df1['Date']=df1.index.day
    df1['Day']=df1.index.weekday
    df1['hour']=df1.index.hour
    
    # Season 1: 봄 ~ 4: 겨울 
    df1['season'] = df1.Mth.apply(lambda x: 4 if x in [12,1,2] else 
                1 if x in [3,4,5] else 2 if x in [6,7,8]
               else 3)
    # dayClass 1: 월, 2: 화~금, 3: 토, 일 
    # 참조 from 실시간 수요예측 기법 개발 및 온라인 수요예측 방안 연구_KPX 한국전력거래소)
    df1['day'] = df1.Day.apply(lambda x: 1 if x ==0 else 
                2 if x in [1,2,3,4] else 3)
    return df1

In [314]:
TE_time_vars = dateTimeVar(test)

In [315]:
TE_time_vars.head(5)

,Mth,Date,Day,hour,season,day
Time,,,,,,
2017-08-29 10:00:00,8,29,1,10,2,2
2017-08-29 11:00:00,8,29,1,11,2,2
2017-08-29 12:00:00,8,29,1,12,2,2
2017-08-29 13:00:00,8,29,1,13,2,2
2017-08-29 14:00:00,8,29,1,14,2,2


## Make DUMMIES

In [316]:
test.head(2)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.114,0.087,0.283,0.342142,1.148489,0.094,0.351,1.109,0.225,0.29667
2017-08-29 11:00:00,1.458903,0.398,0.293,0.358,0.350,0.232,0.253,0.260,0.168,0.084,...,0.163,0.086,0.288,0.342010,1.150483,0.196,0.343,1.109,0.225,0.29667


In [317]:
TE_time_vars.head(2)

,Mth,Date,Day,hour,season,day
Time,,,,,,
2017-08-29 10:00:00,8,29,1,10,2,2
2017-08-29 11:00:00,8,29,1,11,2,2


In [318]:
def dummies(df, dum_lst):
    df1 = df.copy()
    # dum_lst에 들어있는 각각의 변수마다 
    # tmp = get_dummies, 해당 original 변수는 drop
    # tmp와 변수가 drop된 df를 join 후 리턴
    for v in dum_lst:
        tmp = pd.get_dummies(df1[v], prefix=v)
        df1 = df1.drop(columns=v).join(tmp)
    return df1

In [319]:
TE_timeV_dum = dummies(TE_time_vars, ['season', 'day'])

In [320]:
TE_timeV_dum_wthr= pd.concat([TE_timeV_dum, w_hr2], axis=1,join='inner')

In [321]:
TE_timeV_dum_wthr.head(10)

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,day_1,day_2,day_3,Temp,Humid,Wind
Time,,,,,,,,,,,,,,
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0,19.3,0.0,5.0
2017-08-29 11:00:00,8,29,1,11,0,1,0,0,0,1,0,19.7,0.0,4.7
2017-08-29 12:00:00,8,29,1,12,0,1,0,0,0,1,0,19.4,0.0,4.8
2017-08-29 13:00:00,8,29,1,13,0,1,0,0,0,1,0,19.8,0.0,4.5
2017-08-29 14:00:00,8,29,1,14,0,1,0,0,0,1,0,20.5,0.0,3.4
2017-08-29 15:00:00,8,29,1,15,0,1,0,0,0,1,0,20.6,0.0,4.0
2017-08-29 16:00:00,8,29,1,16,0,1,0,0,0,1,0,22.2,0.0,5.2
2017-08-29 17:00:00,8,29,1,17,0,1,0,0,0,1,0,21.7,0.0,5.5
2017-08-29 18:00:00,8,29,1,18,0,1,0,0,0,1,0,21.2,0.0,5.3


# Modeling

* test : TEST셋의 순수한 meterIDs Time series (NA값 채워짐) 
* TE_timeV_dum: TEST셋의 시간파생변수 + dummy variables 

In [322]:
test.head(1)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.114,0.087,0.283,0.342142,1.148489,0.094,0.351,1.109,0.225,0.29667


In [323]:
TE_timeV_dum_wthr.head(1)

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,day_1,day_2,day_3,Temp,Humid,Wind
Time,,,,,,,,,,,,,,
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0,19.3,0.0,5.0


<br><br><br>


## Random sample 된 30개만 돌리기 (06.20~30 예측)

In [324]:
rs = pd.read_csv("data/train_all.csv", index_col=0, sep=',')
ids = rs.columns

In [325]:
test = pd.read_csv("data/TEST_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')
train = pd.read_csv("data/TRAIN_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')

In [326]:
te = ['X41','X50','X30', 'X37', 'X18','X231', 'X116', 'X149', 'X371', 'X218']
tr = ['X40', 'X84' ,'X77', 'X93', 'X89' ,'X15', 'X23', 'X60', 'X27', 'X6','X610' ,'X1362' ,'X476', 'X237' ,'X1032', 'X1287', 'X428', 'X768', 'X989' ,'X616']

In [327]:
len(te)

10

In [328]:
test.loc[:,te].head(2)

,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,
2017-08-29 10:00:00,2.167430,4.171678,3.498120,3.816439,2.518855,0.216611,0.602,0.255,0.214,0.332322
2017-08-29 11:00:00,2.159356,4.122120,3.461189,3.801315,2.519404,0.216574,0.602,0.428,0.256,0.332338


In [329]:
train.loc[:,tr]

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,X610,X1362,X476,X237,X1032,X1287,X428,X768,X989,X616
Time,,,,,,,,,,,,,,,,,,,,
2018-02-15 00:00:00,10.592437,0.164,0.561,0.340,1.396,2.284564,2.691859,2.094527,4.322733,4.749095,0.066000,0.132000,0.091000,0.318000,0.277000,0.096000,0.020000,1.339000,0.113000,0.351000
2018-02-15 01:00:00,10.592437,0.161,0.486,0.344,1.394,2.284564,2.691859,2.094527,4.322733,4.749095,0.126000,0.278000,0.244000,0.202000,0.474000,0.338000,0.206000,0.599000,0.178000,1.495000
2018-02-15 02:00:00,10.592437,0.123,0.569,0.350,1.407,2.284564,2.691859,2.094527,4.322733,4.749095,0.227000,0.282000,0.252000,0.190000,0.302000,0.222000,0.134000,0.515000,0.223000,0.859000
2018-02-15 03:00:00,10.592437,0.115,0.590,0.333,1.396,2.284564,2.691859,2.094527,4.322733,4.749095,0.125000,0.286000,0.254000,0.163000,0.349000,0.282000,0.223000,0.458000,0.186000,0.492000
2018-02-15 04:00:00,10.592437,0.130,0.519,0.345,1.401,2.284564,2.691859,2.094527,4.322733,4.749095,0.135000,0.266000,0.251000,0.196000,0.305000,0.257000,0.266000,0.415000,0.212000,0.471000
2018-02-15 05:00:00,10.592437,0.113,0.517,0.347,1.396,2.284564,2.691859,2.094527,4.322733,4.749095,0.087000,0.292000,0.241000,0.198000,0.284000,0.258000,0.083000,0.475000,0.207000,0.374000
2018-02-15 06:00:00,10.592437,0.116,0.501,0.342,1.409,2.284564,2.691859,2.094527,4.322733,4.749095,0.104000,0.288000,0.423000,0.183000,0.313000,0.215000,0.089000,0.564000,0.290000,0.315000
2018-02-15 07:00:00,10.592437,0.114,0.556,0.337,1.395,2.284564,2.691859,2.094527,4.322733,4.749095,0.568000,0.615000,0.665000,0.150000,0.500000,0.218000,0.101000,0.479000,0.320000,0.355000
2018-02-15 08:00:00,10.592437,0.131,0.539,0.345,1.382,2.284564,2.691859,2.094527,4.322733,4.749095,0.677000,0.402000,0.916000,0.193000,0.352000,0.253000,0.335000,0.563000,0.828000,1.154000


In [330]:
test.loc['2018-02-15 00:00:00':,te]

,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,
2018-02-15 00:00:00,2.371049,6.545218,4.803182,4.499756,2.495460,0.327000,0.199,0.261000,0.142000,0.361000
2018-02-15 01:00:00,2.334381,5.984280,4.529291,4.341709,2.500658,0.202000,0.168,0.391000,0.319000,0.241000
2018-02-15 02:00:00,2.302862,5.544196,4.305131,4.216720,2.504838,0.213000,0.123,0.248000,0.306000,0.284000
2018-02-15 03:00:00,2.275769,5.198929,4.121671,4.117873,2.508198,0.154000,0.113,0.215000,0.280000,0.295000
2018-02-15 04:00:00,2.252481,4.928049,3.971522,4.039701,2.510895,0.141000,0.167,0.211000,0.294000,0.267000
2018-02-15 05:00:00,2.232463,4.715530,3.848635,3.977879,2.513058,0.154000,0.182,0.219000,0.433000,0.347000
2018-02-15 06:00:00,2.215256,4.548798,3.748061,3.928988,2.514790,0.150000,0.226,0.214000,0.300000,0.706000
2018-02-15 07:00:00,2.200466,4.417989,3.665748,3.890323,2.516175,0.186000,0.115,0.266000,0.418000,0.514000
2018-02-15 08:00:00,2.187752,4.315363,3.598380,3.859746,2.517280,0.134000,0.115,0.404000,0.400000,0.675000


In [331]:
data = pd.concat([train.loc[:, tr], test.loc['2018-02-15 00:00:00':,te]], axis=1 )
data.index = pd.to_datetime(data.index)

c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [377]:
hourly = pd.read_csv("data/TRAIN_DHWonDacon_24시간예측.csv",index_col=0)
hourly.head(2)

,X692,X1272,X553,X1299,X598,X1003,X1010,X1216,X1047,X381,...,X1164,X656,X1287,X324,X767,X1271,X1288,X58,X1033,X1163
1,0.374392,0.444582,0.577257,0.239870,0.358361,0.523760,0.634239,0.356952,0.265332,0.234563,...,0.440065,0.421038,0.329439,0.351530,0.578465,0.649527,0.640117,7.835084,0.331562,0.371070
2,0.366835,0.422568,0.556617,0.211252,0.310659,0.447532,0.482711,0.331569,0.265641,0.213087,...,0.416521,0.426182,0.321118,0.289726,0.486627,0.600352,0.603710,7.066954,0.338846,0.350562


In [378]:
data2 = data.copy()

In [379]:
data3 = pd.DataFrame()   # 새로운 데이터프레임 생성
n=0  # 현재 칼럼카운트를 보여주기 위한 변수

# 미터ID 리스트 c에서 1개씩 for loop
for i in data2.columns:
    if i in hourly.columns:
        n +=1   # 카운트 
        li = data2[data2[i].isnull()].index   # li: test데이터프레임의 특정 열(i)에서 Null인 셀 index추출 
        df = pd.DataFrame(data2[i])    # df: test의 i열을 별개 데이터프레임으로 생성
        print("%d. current column: %s, NaN count: %d" %(n, i, len(li)))   
        for t in li:    #  li(Null 인덱스)를 사용해 for loop
            df.loc[t] = hourly.loc[:,i].iloc[t.hour]  # null값에 t.hour 평균 대입
        data3 = pd.concat([data3,df], axis=1)   # 평균 대치가 끝난 df를 test2에 concat해준다. 

1. current column: X40, NaN count: 1507
2. current column: X84, NaN count: 0
3. current column: X77, NaN count: 0
4. current column: X93, NaN count: 0
5. current column: X89, NaN count: 0
6. current column: X15, NaN count: 1507
7. current column: X23, NaN count: 1507
8. current column: X60, NaN count: 1507
9. current column: X27, NaN count: 1507
10. current column: X6, NaN count: 1507
11. current column: X610, NaN count: 0
12. current column: X1362, NaN count: 0
13. current column: X476, NaN count: 0
14. current column: X237, NaN count: 0
15. current column: X1032, NaN count: 0
16. current column: X1287, NaN count: 0
17. current column: X428, NaN count: 0
18. current column: X768, NaN count: 0
19. current column: X989, NaN count: 0
20. current column: X616, NaN count: 0


In [380]:
train_data = pd.concat([data3, test.loc['2018-02-15 00:00:00':,te]], axis=1 )
train_data.index = pd.to_datetime(train_data.index)
train_data.head(3)

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,...,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,,,,,,,,,,,,
2018-02-15 00:00:00,15.991599,0.164,0.561,0.340,1.396,3.585059,0.516354,8.630316,1.511542,6.340007,...,2.371049,6.545218,4.803182,4.499756,2.495460,0.327,0.199,0.261,0.142,0.361
2018-02-15 01:00:00,14.878767,0.161,0.486,0.344,1.394,3.391193,0.948824,7.226755,1.918634,6.018741,...,2.334381,5.984280,4.529291,4.341709,2.500658,0.202,0.168,0.391,0.319,0.241
2018-02-15 02:00:00,13.994133,0.123,0.569,0.350,1.407,3.225439,1.295868,6.123420,2.268595,5.761976,...,2.302862,5.544196,4.305131,4.216720,2.504838,0.213,0.123,0.248,0.306,0.284


In [381]:
train_data.isnull().sum()

X40      0
X84      0
X77      0
X93      0
X89      0
X15      0
X23      0
X60      0
X27      0
X6       0
X610     0
X1362    0
X476     0
X237     0
X1032    0
X1287    0
X428     0
X768     0
X989     0
X616     0
X41      0
X50      0
X30      0
X37      0
X18      0
X231     0
X116     0
X149     0
X371     0
X218     0
dtype: int64

In [382]:
train = train_data.loc[:'2018-06-29 23:00:00', :]
train.tail()

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,...,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,,,,,,,,,,,,
2018-06-29 19:00:00,15.143,1.039,0.147,0.343,5.752,3.987,8.032,1.597,9.358,6.870,...,2.318,12.638,13.212,7.407,2.524,0.308706,0.206,0.475391,0.483082,0.474253
2018-06-29 20:00:00,15.558,2.576,0.135,0.409,5.412,3.990,8.588,5.206,7.631,6.875,...,2.236,10.939,12.781,9.560,2.508,0.379719,0.177,0.729524,0.433335,0.965936
2018-06-29 21:00:00,14.979,3.832,0.091,0.363,3.621,3.984,7.941,8.735,8.224,6.591,...,2.275,7.216,9.197,7.907,2.525,0.413473,0.125,0.669142,0.517561,0.855677
2018-06-29 22:00:00,16.243,3.532,0.086,0.342,1.455,3.965,0.503,5.614,3.781,6.955,...,2.370,6.527,5.359,12.199,2.469,0.470807,0.108,0.603334,0.488104,0.566388
2018-06-29 23:00:00,16.655,4.643,0.086,0.394,1.427,3.870,0.135,5.493,2.112,6.783,...,2.607,6.577,3.251,11.825,2.501,0.092000,0.172,0.085000,0.115000,0.142000


In [383]:
train.isnull().sum()

X40      0
X84      0
X77      0
X93      0
X89      0
X15      0
X23      0
X60      0
X27      0
X6       0
X610     0
X1362    0
X476     0
X237     0
X1032    0
X1287    0
X428     0
X768     0
X989     0
X616     0
X41      0
X50      0
X30      0
X37      0
X18      0
X231     0
X116     0
X149     0
X371     0
X218     0
dtype: int64

In [384]:
features = TE_timeV_dum_wthr.loc['2018-02-15 00:00:00':'2018-06-29 23:00:00', :]
features.head()

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,day_1,day_2,day_3,Temp,Humid,Wind
Time,,,,,,,,,,,,,,
2018-02-15 00:00:00,2,15,3,0,0,0,0,1,0,1,0,-1.0,0.0,5.0
2018-02-15 01:00:00,2,15,3,1,0,0,0,1,0,1,0,-1.2,0.0,3.4
2018-02-15 02:00:00,2,15,3,2,0,0,0,1,0,1,0,-1.2,0.0,3.9
2018-02-15 03:00:00,2,15,3,3,0,0,0,1,0,1,0,-1.5,0.0,3.5
2018-02-15 04:00:00,2,15,3,4,0,0,0,1,0,1,0,-1.9,0.0,5.3


In [385]:
test_real = train_data.loc['2018-06-30 00:00:00':, :]
test_real

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,...,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,,,,,,,,,,,,
2018-06-30 00:00:00,16.819,2.255,0.086,0.401,1.336,2.340,0.172,6.566,1.032,6.962,...,2.606,6.493,1.940,10.739,2.535,0.252348,0.210,0.363205,0.435221,0.322871
2018-06-30 01:00:00,16.731,0.175,0.102,0.369,1.355,1.313,0.420,5.542,0.934,6.751,...,2.736,6.189,0.395,6.727,2.540,0.236122,0.229,0.345738,0.394364,0.327677
2018-06-30 02:00:00,14.620,0.220,0.100,0.345,1.383,1.314,0.407,0.882,0.982,6.336,...,2.652,4.934,0.386,4.320,2.509,0.085000,0.204,0.146000,0.126000,0.265000
2018-06-30 03:00:00,9.068,0.238,0.102,0.379,1.338,1.314,0.255,1.036,0.894,6.437,...,2.639,5.839,0.392,4.743,2.518,0.143000,0.176,0.272000,0.264000,0.585000
2018-06-30 04:00:00,3.512,0.238,0.086,0.402,1.352,1.314,0.207,1.156,1.017,6.637,...,2.728,5.366,0.427,2.764,2.534,0.146000,0.219,0.215000,0.241000,0.861000
2018-06-30 05:00:00,3.445,0.151,0.085,0.350,1.338,1.313,0.133,0.938,0.945,5.968,...,2.564,5.264,0.376,2.119,2.610,0.153000,0.228,0.240000,0.244000,0.572000
2018-06-30 06:00:00,3.429,0.160,0.085,0.356,1.367,1.314,0.392,0.697,0.939,5.529,...,0.965,5.000,0.375,2.142,2.812,0.134000,0.174,0.235000,0.303000,0.416000
2018-06-30 07:00:00,4.437,0.196,0.085,0.320,1.358,1.313,0.427,1.917,0.925,5.047,...,0.420,4.464,0.376,2.319,2.665,0.163000,0.189,0.469000,0.285000,0.446000
2018-06-30 08:00:00,8.780,0.220,0.096,0.366,1.351,2.314,0.882,0.888,1.329,5.059,...,0.430,3.544,0.363,2.109,2.617,0.170000,0.109,0.311000,0.382000,0.276000


In [386]:
X_test = TE_timeV_dum_wthr.loc['2018-06-30 00:00:00':, :]
X_test.head()

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,day_1,day_2,day_3,Temp,Humid,Wind
Time,,,,,,,,,,,,,,
2018-06-30 00:00:00,6,30,5,0,0,1,0,0,0,0,1,20.1,0.0,0.0
2018-06-30 01:00:00,6,30,5,1,0,1,0,0,0,0,1,20.2,0.0,0.0
2018-06-30 02:00:00,6,30,5,2,0,1,0,0,0,0,1,19.7,0.0,0.0
2018-06-30 03:00:00,6,30,5,3,0,1,0,0,0,0,1,19.7,0.0,0.0
2018-06-30 04:00:00,6,30,5,4,0,1,0,0,0,0,1,19.6,0.0,0.0


In [387]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

lg_param = {"boosting_type":['gbdt', 'dart','goss'],
    "learning_rate": [0.01,0.05,0.1,0.15],
    "min_child_samples":[10, 20, 30, 50],       
    "n_estimators":[10, 50, 100],
    }

def gsLGBR_Hrly_FitPred (model, params, mID, X_train, y_train, X_test):
    grid_gb = GridSearchCV(model, params, cv=4, verbose=5)
    grid_gb.fit(X_train, y_train)
    fcst = grid_gb.predict(X_test)
    a = pd.DataFrame([mID], columns=['meter_id'])
    for i in range(240):
        a[i+1]=[fcst[i]]
    return a
   
def onlytesting(train, features, X_test):
    hrly = pd.DataFrame()
    for mID in train.columns:
        data = pd.concat([train.loc[:,[mID]], features], axis=1)
        data = data.rename(columns={mID:'y'})
        for i in [24, 168]:
            data['lag_{}'.format(i)] = data.y.shift(i)
        y_train = data.dropna().y
        X_train = data.dropna().drop(['y'], axis=1)
        
        X_test['lag_24'] = y_train.tail(24).values
        X_test['lag_168'] = data.y.loc['2018-06-23 00:00:00':'2018-06-23 23:00:00'].values
        
        model = LGBMRegressor(verbose=1, tree_learner='feature')
        tmp = gsLGBR_Hrly_FitPred(model, lg_param,
                        mID, X_train, y_train, X_test_H)
        hrly = pd.concat([hrly_50, tmp], axis=0)
    return hrly

ModuleNotFoundError: No module named 'lightgbm'

In [388]:
def smape_fast(df_y_true, df_y_pred):
    """
    @jit
    using SMAPE(Symmetric mean absolute percentage error)
    :setting:install 'numba' in anaconda
    :df_y_true: Real DateFrame
    :df_y_pred: DataFrame you made    
    """
    from numba import jit
    import math
    out = 0
    if len(df_y_true.columns) > 1:
        y_true = df_y_true.drop(df_y_true.columns[0], axis=1).values.reshape(-1)
        y_pred = df_y_pred.drop(df_y_pred.columns[0], axis=1).values.reshape(-1)
    else:
        y_true = df_y_true.values.reshape(-1)
        y_pred = df_y_pred.values.reshape(-1)
    for i in range(y_true.shape[0]):
        a = y_true[i]
        b = y_pred[i]
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

# Random Forest

In [345]:
from tqdm import tqdm_notebook

In [346]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

def makeTrainTest (mID, train, features, X_test):
    data = pd.concat([train.loc[:,[mID]], features], axis=1)
    data = data.rename(columns={mID:'y'})
    for i in [24, 168]:
        data['lag_{}'.format(i)] = data.y.shift(i)
    y_train = data.dropna().y
    X_train = data.dropna().drop(['y'], axis=1)

    X_test['lag_24'] = data.y.loc['2018-06-29 00:00:00':'2018-06-29 23:00:00'].values
    X_test['lag_168'] = data.y.loc['2018-06-23 00:00:00':'2018-06-23 23:00:00'].values
    
    return X_train, y_train, X_test

def onlytesting(train, features, X_test):
    hrly = pd.DataFrame()
    for mID in tqdm_notebook(train.columns):
        X_train, y_train, X_test = makeTrainTest(mID, train, features, X_test)
        
        model_LR = LinearRegression()
        model_LR.fit(X_train, y_train)
        fcst = model_LR.predict(X_test)
        
        tmp = pd.DataFrame([mID], columns=['meter_id'])
        for i in range(24):
            tmp[i+1]=[fcst[i]]
        
#         model = LGBMRegressor(verbose=1, tree_learner='feature')
#         tmp = gsLGBR_Hrly_FitPred(model, lg_param,
#                         mID, X_train, y_train, X_test_H)
        hrly = pd.concat([hrly, tmp], axis=0)
    return hrly

In [347]:
train.head(2)

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,...,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
2018-02-15 00:00:00,10.592437,0.164,0.561,0.340,1.396,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.371049,6.545218,4.803182,4.499756,2.495460,0.327,0.199,0.261,0.142,0.361
2018-02-15 01:00:00,10.592437,0.161,0.486,0.344,1.394,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.334381,5.984280,4.529291,4.341709,2.500658,0.202,0.168,0.391,0.319,0.241


In [348]:
features.shape

(3240, 14)

In [349]:
X_test.shape

(24, 14)

In [352]:
LR_test = onlytesting(train, features, X_test)

c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [351]:
LR_test = LR_test.set_index('meter_id')

In [286]:
smape_fast(test_real.T, LR_test)

43.70829386490359

### Day vs. Day_1,2,3

In [259]:
TE_timeV_dum_wthr2 = TE_timeV_dum_wthr.drop(columns='Day')

In [260]:
features_noDay = TE_timeV_dum_wthr2.loc['2018-02-15 00:00:00':'2018-06-29 23:00:00', :]
features_noDay.head()

,Mth,Date,hour,season_1,season_2,season_3,season_4,day_1,day_2,day_3,Temp,Humid,Wind
Time,,,,,,,,,,,,,
2018-02-15 00:00:00,2,15,0,0,0,0,1,0,1,0,-1.0,0.0,5.0
2018-02-15 01:00:00,2,15,1,0,0,0,1,0,1,0,-1.2,0.0,3.4
2018-02-15 02:00:00,2,15,2,0,0,0,1,0,1,0,-1.2,0.0,3.9
2018-02-15 03:00:00,2,15,3,0,0,0,1,0,1,0,-1.5,0.0,3.5
2018-02-15 04:00:00,2,15,4,0,0,0,1,0,1,0,-1.9,0.0,5.3


In [261]:
X_test_noDay = X_test.drop(columns='Day')

In [262]:
RF_test_noDay = onlytesting(train, features_noDay, X_test_noDay)

c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

In [263]:
RF_test_noDay.head(2)

,meter_id,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,X40,14.382,14.457477,12.34837,11.0195,4.0473,3.2639,3.1023,4.3088,12.660049,...,19.3601,19.5118,17.9355,17.7582,19.2023,18.6562,17.0090,16.421600,16.195956,16.089156
0,X84,0.204,1.084700,1.75510,1.7574,1.7649,0.2426,0.6127,0.1800,0.522700,...,4.3588,5.1502,4.3911,4.8650,4.3515,4.6025,3.8054,2.895449,2.121819,2.194400


In [264]:
RF_test_noDay = RF_test_noDay.set_index('meter_id')

In [265]:
smape_fast(test_real.T, RF_test_noDay)

33.83599812956374

In [266]:
TE_timeV_dum_wthr3 = TE_timeV_dum_wthr.drop(columns=['day_1', 'day_2','day_3'])

In [267]:
features_noDayClass = TE_timeV_dum_wthr3.loc['2018-02-15 00:00:00':'2018-06-29 23:00:00', :]
features_noDayClass.head()

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,Temp,Humid,Wind
Time,,,,,,,,,,,
2018-02-15 00:00:00,2,15,3,0,0,0,0,1,-1.0,0.0,5.0
2018-02-15 01:00:00,2,15,3,1,0,0,0,1,-1.2,0.0,3.4
2018-02-15 02:00:00,2,15,3,2,0,0,0,1,-1.2,0.0,3.9
2018-02-15 03:00:00,2,15,3,3,0,0,0,1,-1.5,0.0,3.5
2018-02-15 04:00:00,2,15,3,4,0,0,0,1,-1.9,0.0,5.3


In [268]:
X_test_noDayClass = X_test.drop(columns=['day_1', 'day_2','day_3'])

In [269]:
RF_test_noDayClass = onlytesting(train, features_noDay, X_test_noDay)

c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

In [270]:
RF_test_noDayClass = RF_test_noDayClass.set_index('meter_id')

In [271]:
smape_fast(test_real.T, RF_test_noDayClass)

34.05389342514043

## Random 30 columns

In [274]:
clusters = pd.read_csv("presentation/clusterIDs_k2.csv", index_col=0, sep=',')

In [277]:
clusters

,0,1,2,3,4,5,6,7,8,9,...,1259,1260,1261,1262,1263,1264,1265,1266,1267,1268
TEST_Cluster1,X10,X102,X105,X110,X116,X120,X124,X125,X16,X30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TEST_Cluster2,X106,X107,X111,X123,X126,X130,X132,X133,X134,X139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TRAIN_Cluster1,X101,X103,X104,X113,X119,X121,X15,X23,X27,X28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TRAIN_Cluster2,X1,X100,X1000,X1001,X1002,X1003,X1004,X1005,X1006,X1007,...,X990,X991,X992,X993,X994,X995,X996,X997,X998,X999


In [278]:
sum(clusters.isnull().sum())

3576

In [287]:
test = pd.read_csv("data/test.csv", index_col=0, sep=',')

In [288]:
test.X303.median()

0.434

In [289]:
test.X241.median()

0.321

In [290]:
test.X435.median()

0.312

In [292]:
test.X402.median()

0.271

In [293]:
test.X352.median()

0.2

In [294]:
test.X305.median()

0.3670000000000001

In [299]:
test.X350.median()

0.311

In [297]:
test.X326.median()

0.08

In [298]:
test.X299.median()

0.239

## 언니 Rolling

In [ ]:
rolling_test = pd.read_csv("data/test_r48_med_onlyNAN.csv", index_col=0, sep=',')
rolling_train = pd.read_csv("data/train_r48_med_onlyNAN.csv", index_col=0, sep=',')

In [ ]:
te = ['X41','X50','X30', 'X37', 'X18','X231', 'X116', 'X149', 'X371', 'X218']
tr = ['X40', 'X84' ,'X77', 'X93', 'X89' ,'X15', 'X23', 'X60', 'X27', 'X6','X610' ,'X1362' ,'X476', 'X237' ,'X1032', 'X1287', 'X428', 'X768', 'X989' ,'X616']

In [365]:
rolling_test.loc[:,te].head(2)

,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,
2017-07-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,0.243,NaN,NaN,NaN
2017-07-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,0.299,NaN,NaN,NaN


In [366]:
rolling_train.loc[:,tr]

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,X610,X1362,X476,X237,X1032,X1287,X428,X768,X989,X616
Time,,,,,,,,,,,,,,,,,,,,
2016-07-26 11:00:00,NaN,NaN,3.815,3.921,2.005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 12:00:00,NaN,NaN,3.196,5.233,3.674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 13:00:00,NaN,NaN,5.576,5.176,3.630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 14:00:00,NaN,NaN,5.101,4.979,3.678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 15:00:00,NaN,NaN,5.063,4.898,3.693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 16:00:00,NaN,NaN,4.762,4.206,5.522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 17:00:00,NaN,NaN,4.633,4.115,7.893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 18:00:00,NaN,NaN,4.672,4.020,5.137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 19:00:00,NaN,NaN,3.587,3.070,1.457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [367]:
rolling_test.loc['2018-02-15 00:00:00':,te]

,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,
2018-02-15 00:00:00,NaN,NaN,NaN,NaN,NaN,0.327,0.199,0.261,0.142,0.361
2018-02-15 01:00:00,NaN,NaN,NaN,NaN,NaN,0.202,0.168,0.391,0.319,0.241
2018-02-15 02:00:00,NaN,NaN,NaN,NaN,NaN,0.213,0.123,0.248,0.306,0.284
2018-02-15 03:00:00,NaN,NaN,NaN,NaN,NaN,0.154,0.113,0.215,0.280,0.295
2018-02-15 04:00:00,NaN,NaN,NaN,NaN,NaN,0.141,0.167,0.211,0.294,0.267
2018-02-15 05:00:00,NaN,NaN,NaN,NaN,NaN,0.154,0.182,0.219,0.433,0.347
2018-02-15 06:00:00,NaN,NaN,NaN,NaN,NaN,0.150,0.226,0.214,0.300,0.706
2018-02-15 07:00:00,NaN,NaN,NaN,NaN,NaN,0.186,0.115,0.266,0.418,0.514
2018-02-15 08:00:00,NaN,NaN,NaN,NaN,NaN,0.134,0.115,0.404,0.400,0.675


In [371]:
data = pd.concat([rolling_train.loc['2018-02-15 00:00:00':, tr], rolling_test.loc['2018-02-15 00:00:00':,te]], axis=1 )
data.index = pd.to_datetime(data.index)
data.head(2)

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,...,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,,,,,,,,,,,,
2018-02-15 00:00:00,NaN,0.164,0.561,0.340,1.396,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.327,0.199,0.261,0.142,0.361
2018-02-15 01:00:00,NaN,0.161,0.486,0.344,1.394,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.202,0.168,0.391,0.319,0.241


In [372]:
data.isnull().sum()

X40      1507
X84         0
X77         0
X93         0
X89         0
X15      1507
X23      1507
X60      1507
X27      1507
X6       1507
X610        0
X1362       0
X476        0
X237        0
X1032       0
X1287       0
X428        0
X768        0
X989        0
X616        0
X41      1507
X50      1507
X30      1507
X37      1507
X18      1507
X231      204
X116        0
X149      207
X371      195
X218      207
dtype: int64

In [373]:
train = data.loc[:'2018-06-29 23:00:00', :]
train.tail()

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,...,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,,,,,,,,,,,,
2018-06-29 19:00:00,15.143,1.039,0.147,0.343,5.752,3.987,8.032,1.597,9.358,6.870,...,2.318,12.638,13.212,7.407,2.524,NaN,0.206,NaN,NaN,NaN
2018-06-29 20:00:00,15.558,2.576,0.135,0.409,5.412,3.990,8.588,5.206,7.631,6.875,...,2.236,10.939,12.781,9.560,2.508,1.264,0.177,2.002,1.439,2.388
2018-06-29 21:00:00,14.979,3.832,0.091,0.363,3.621,3.984,7.941,8.735,8.224,6.591,...,2.275,7.216,9.197,7.907,2.525,NaN,0.125,NaN,NaN,NaN
2018-06-29 22:00:00,16.243,3.532,0.086,0.342,1.455,3.965,0.503,5.614,3.781,6.955,...,2.370,6.527,5.359,12.199,2.469,NaN,0.108,NaN,NaN,NaN
2018-06-29 23:00:00,16.655,4.643,0.086,0.394,1.427,3.870,0.135,5.493,2.112,6.783,...,2.607,6.577,3.251,11.825,2.501,0.092,0.172,0.085,0.115,0.142


In [338]:
train.isnull().sum()

X40      0
X84      0
X77      0
X93      0
X89      0
X15      0
X23      0
X60      0
X27      0
X6       0
X610     0
X1362    0
X476     0
X237     0
X1032    0
X1287    0
X428     0
X768     0
X989     0
X616     0
X41      0
X50      0
X30      0
X37      0
X18      0
X231     0
X116     0
X149     0
X371     0
X218     0
dtype: int64

In [374]:
features = TE_timeV_dum_wthr.loc['2018-02-15 00:00:00':'2018-06-29 23:00:00', :]
features.head()

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,day_1,day_2,day_3,Temp,Humid,Wind
Time,,,,,,,,,,,,,,
2018-02-15 00:00:00,2,15,3,0,0,0,0,1,0,1,0,-1.0,0.0,5.0
2018-02-15 01:00:00,2,15,3,1,0,0,0,1,0,1,0,-1.2,0.0,3.4
2018-02-15 02:00:00,2,15,3,2,0,0,0,1,0,1,0,-1.2,0.0,3.9
2018-02-15 03:00:00,2,15,3,3,0,0,0,1,0,1,0,-1.5,0.0,3.5
2018-02-15 04:00:00,2,15,3,4,0,0,0,1,0,1,0,-1.9,0.0,5.3


In [398]:
test_real = train_data.loc['2018-06-30 00:00:00':, :]
test_real

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,...,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,,,,,,,,,,,,
2018-06-30 00:00:00,16.819,2.255,0.086,0.401,1.336,2.340,0.172,6.566,1.032,6.962,...,2.606,6.493,1.940,10.739,2.535,0.252348,0.210,0.363205,0.435221,0.322871
2018-06-30 01:00:00,16.731,0.175,0.102,0.369,1.355,1.313,0.420,5.542,0.934,6.751,...,2.736,6.189,0.395,6.727,2.540,0.236122,0.229,0.345738,0.394364,0.327677
2018-06-30 02:00:00,14.620,0.220,0.100,0.345,1.383,1.314,0.407,0.882,0.982,6.336,...,2.652,4.934,0.386,4.320,2.509,0.085000,0.204,0.146000,0.126000,0.265000
2018-06-30 03:00:00,9.068,0.238,0.102,0.379,1.338,1.314,0.255,1.036,0.894,6.437,...,2.639,5.839,0.392,4.743,2.518,0.143000,0.176,0.272000,0.264000,0.585000
2018-06-30 04:00:00,3.512,0.238,0.086,0.402,1.352,1.314,0.207,1.156,1.017,6.637,...,2.728,5.366,0.427,2.764,2.534,0.146000,0.219,0.215000,0.241000,0.861000
2018-06-30 05:00:00,3.445,0.151,0.085,0.350,1.338,1.313,0.133,0.938,0.945,5.968,...,2.564,5.264,0.376,2.119,2.610,0.153000,0.228,0.240000,0.244000,0.572000
2018-06-30 06:00:00,3.429,0.160,0.085,0.356,1.367,1.314,0.392,0.697,0.939,5.529,...,0.965,5.000,0.375,2.142,2.812,0.134000,0.174,0.235000,0.303000,0.416000
2018-06-30 07:00:00,4.437,0.196,0.085,0.320,1.358,1.313,0.427,1.917,0.925,5.047,...,0.420,4.464,0.376,2.319,2.665,0.163000,0.189,0.469000,0.285000,0.446000
2018-06-30 08:00:00,8.780,0.220,0.096,0.366,1.351,2.314,0.882,0.888,1.329,5.059,...,0.430,3.544,0.363,2.109,2.617,0.170000,0.109,0.311000,0.382000,0.276000


In [399]:
X_test = TE_timeV_dum_wthr.loc['2018-06-30 00:00:00':, :]
X_test.head()

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,day_1,day_2,day_3,Temp,Humid,Wind
Time,,,,,,,,,,,,,,
2018-06-30 00:00:00,6,30,5,0,0,1,0,0,0,0,1,20.1,0.0,0.0
2018-06-30 01:00:00,6,30,5,1,0,1,0,0,0,0,1,20.2,0.0,0.0
2018-06-30 02:00:00,6,30,5,2,0,1,0,0,0,0,1,19.7,0.0,0.0
2018-06-30 03:00:00,6,30,5,3,0,1,0,0,0,0,1,19.7,0.0,0.0
2018-06-30 04:00:00,6,30,5,4,0,1,0,0,0,0,1,19.6,0.0,0.0


In [400]:
r_LR = onlytesting(train, features, X_test)

c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [401]:
smape_fast(test_real.T,r_LR)

63.696314969255376

## 모델들 테스팅

### LGB

In [402]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

lg_param = {"boosting_type":['gbdt', 'dart','goss'],
    "learning_rate": [0.01,0.05,0.1,0.15],
    "min_child_samples":[10, 20, 30, 50],       
    "n_estimators":[10, 50, 100],
    }

def gsLGBR_Hrly_FitPred (model, params, mID, X_train, y_train, X_test):
    grid_gb = GridSearchCV(model, params, cv=4, verbose=5)
    grid_gb.fit(X_train, y_train)
    fcst = grid_gb.predict(X_test)
    a = pd.DataFrame([mID], columns=['meter_id'])
    for i in range(240):
        a[i+1]=[fcst[i]]
    return a
   
def onlytesting(train, features, X_test):
    hrly = pd.DataFrame()
    for mID in train.columns:
        data = pd.concat([train.loc[:,[mID]], features], axis=1)
        data = data.rename(columns={mID:'y'})
        for i in [24, 168]:
            data['lag_{}'.format(i)] = data.y.shift(i)
        y_train = data.dropna().y
        X_train = data.dropna().drop(['y'], axis=1)
        
        X_test['lag_24'] = y_train.tail(24).values
        X_test['lag_168'] = data.y.loc['2018-06-23 00:00:00':'2018-06-23 23:00:00'].values
        
        model = LGBMRegressor(verbose=1, tree_learner='feature')
        tmp = gsLGBR_Hrly_FitPred(model, lg_param,
                        mID, X_train, y_train, X_test_H)
        hrly = pd.concat([hrly_50, tmp], axis=0)
    return hrly

ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression


def makeTrainTest (mID, train, features, X_test):
    data = pd.concat([train.loc[:,[mID]], features], axis=1)
    data = data.rename(columns={mID:'y'})
    for i in [24, 168]:
        data['lag_{}'.format(i)] = data.y.shift(i)
    y_train = data.dropna().y
    X_train = data.dropna().drop(['y'], axis=1)

    X_test['lag_24'] = data.y.loc['2018-06-29 00:00:00':'2018-06-29 23:00:00'].values
    X_test['lag_168'] = data.y.loc['2018-06-23 00:00:00':'2018-06-23 23:00:00'].values
    
    return X_train, y_train, X_test

def onlytesting(train, features, X_test):
    hrly = pd.DataFrame()
    for mID in tqdm_notebook(train.columns):
        X_train, y_train, X_test = makeTrainTest(mID, train, features, X_test)
        
        model_XG = XGBRegressor()
        model_LR.fit(X_train, y_train)
        fcst = model_LR.predict(X_test)
        
        tmp = pd.DataFrame([mID], columns=['meter_id'])
        for i in range(24):
            tmp[i+1]=[fcst[i]]
        
#         model = LGBMRegressor(verbose=1, tree_learner='feature')
#         tmp = gsLGBR_Hrly_FitPred(model, lg_param,
#                         mID, X_train, y_train, X_test_H)
        hrly = pd.concat([hrly, tmp], axis=0)
    return hrly